In [1]:
from pandas.core.categorical import Categorical
from scipy.sparse import csr_matrix
import numpy as np
import pandas as pd
from xgboost import XGBClassifier

C:\Users\33093\AppData\Local\Continuum\Anaconda2\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [3]:
train_data = pd.read_csv('train_data.csv')
test_data = pd.read_csv('test_data.csv')

In [3]:
print train_data.columns

Index([u'connection_id', u'cont_1', u'cont_2', u'cont_3', u'cont_4', u'cont_5',
       u'cont_6', u'cont_7', u'cont_8', u'cont_9', u'cont_10', u'cont_11',
       u'cont_12', u'cont_13', u'cont_14', u'cont_15', u'cont_16', u'cont_17',
       u'cont_18', u'cat_1', u'cat_2', u'cat_3', u'cat_4', u'cat_5', u'cat_6',
       u'cat_7', u'cat_8', u'cat_9', u'cat_10', u'cat_11', u'cat_12',
       u'cat_13', u'cat_14', u'cat_15', u'cat_16', u'cat_17', u'cat_18',
       u'cat_19', u'cat_20', u'cat_21', u'cat_22', u'cat_23', u'target'],
      dtype='object')


In [4]:
def sparse_dummies(categorical_values):
    categories = Categorical(categorical_values)
    N = len(categorical_values)
    row_numbers = np.arange(N, dtype = np.int)
    ones = np.ones((N,))
    return csr_matrix((ones, (row_numbers, categories.codes)))

In [5]:
sparse_dummies(train_data['target'])

<169307x3 sparse matrix of type '<type 'numpy.float64'>'
	with 169307 stored elements in Compressed Sparse Row format>

# One Hot Encode Categorical Data

In [4]:
from sklearn import preprocessing
from sklearn.preprocessing import OneHotEncoder

In [5]:
X = train_data.values[:,1:19]
Y = train_data.values[:,42]
X_Cat = train_data.values[:,19:42]

In [6]:
print X_Cat.shape, X.shape, Y.shape

(169307L, 23L) (169307L, 18L) (169307L,)


In [7]:
encoded_x = None
for i in range(0, X_Cat.shape[1]):
    le = preprocessing.LabelEncoder()
    le_fit = le.fit(X_Cat[:,i])
    feature = le_fit.transform(X_Cat[:,i])
    feature = feature.reshape(X_Cat.shape[0], 1)
    onehot_en = OneHotEncoder(sparse = False)
    feature = onehot_en.fit_transform(feature) 
    if encoded_x is None:
        encoded_x = feature
    else:
        encoded_x = np.concatenate((encoded_x, feature), axis = 1)

In [8]:
print ("X_Cat Shape : ", encoded_x.shape)
print ("X Shape: ", X.shape)
print type(encoded_x)

('X_Cat Shape : ', (169307L, 1501L))
('X Shape: ', (169307L, 18L))
<type 'numpy.ndarray'>


In [ ]:
x_train = np.concatenate((X, encoded_x), axis = 1)

In [11]:
le_y = preprocessing.LabelEncoder()
y = le_y.fit(Y)
y = le_y.transform(Y)

In [12]:
model = XGBClassifier()
model.fit(x_train, y)

XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=1,
       gamma=0, learning_rate=0.1, max_delta_step=0, max_depth=3,
       min_child_weight=1, missing=None, n_estimators=100, nthread=-1,
       objective='binary:logistic', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=0, silent=True, subsample=1)